## This notebook contains implementation of artificial neural network for classical problem of handwritten digit recognition using plain python without any neural network libraries.
## The dataset is the MNIST dataset with 60000 training examples and 10000 test examples. Each training and test example is the grey-scaled image of 28x28 pixels.
## The network has following tunable parameters:
* Number of hidden layers
* Number of neurons in eah hidden layer
* Cost function "Quadratic" or "Cross-entropy"
* Activation function of the output layer can be "Sigmoid" or "Softmax" (activations of hidden layers is always "Sigmoid")
* Mini-batch size
* Learning rate
* Regularization parameter
## The accuracy on the test set reached 97.5% in the best run with following parameters of the network:
* Two hidden layers, 100 and 50 neurons in the first and second layers respectevely
* Cost function "Cross-entropy"
* Activation of the hidden layers "Sigmoid", activation of the output layer "Softmax"
* Mini-batch size for stochastic gradient descent 10
* Learning rate 0.1
* Regularization parameter 4
* Training over 60 epochs


In [1]:
import numpy as np
import idx2numpy
import matplotlib.pyplot as plt

## Importing MNIST data and converting them to numpy arrays

In [2]:
file_imag_train='train-images.idx3-ubyte'
file_label_train='train-labels.idx1-ubyte'
file_imag_test='t10k-images.idx3-ubyte'
file_label_test='t10k-labels.idx1-ubyte'
data_imag=idx2numpy.convert_from_file(file_imag_train)
data_labels=idx2numpy.convert_from_file(file_label_train)
data_imag_test=idx2numpy.convert_from_file(file_imag_test)
data_labels_test=idx2numpy.convert_from_file(file_label_test)

In [3]:
print("Shape of array containing digit pixels: {0}".format(data_imag.shape))
print("Shape of array containing digit labels: {0}".format(data_labels.shape))

Shape of array containing digit pixels: (60000, 28, 28)
Shape of array containing digit labels: (60000,)


## Draw some digits

In [4]:
# for index in range(15):
#     plt.subplot(3,5,index+1)
#     plt.axis('off')
#     plt.imshow(data_imag[index],cmap=plt.cm.gray_r,interpolation='nearest')

## Reshape 2-dim 28x28 pixels image to 1-dim 784 pixels array

In [5]:
data_imag_reshaped=np.reshape(data_imag, (data_imag.shape[0], data_imag.shape[1]*data_imag.shape[2]))
data_imag_test_reshaped=np.reshape(data_imag_test, (data_imag_test.shape[0], data_imag_test.shape[1]*data_imag_test.shape[2]))
inputs=data_imag_reshaped.T
inputs_test=data_imag_test_reshaped.T

## Initializing network parameters

In [6]:
network_architecture=[inputs.shape[0], 100, 50, 10] # [input nodes, hidden layer1, ..., hidden layer n, output_layer]
cost_function='CrossEntropy' # 'Quadratic' or 'CrossEntropy'
activation_last_layer='Softmax' # 'Sigmoid' or 'Softmax'
mini_batch_size=10 # Size of mini-batch for stochastic gradient descent approach
learning_rate=0.1
regularization=4
epochs=60 # number of epochs to train

## Utility functions

In [7]:
def shuffle_data(x,y): # shuffle data for stochastic gradient descent
    randomize=np.arange(x.shape[1])
    np.random.shuffle(randomize)
    x=x[:,randomize]
    y=y[:,randomize]
    return x, y
def cost_fn(y_true, a, training_examples):
    if cost_function=='Quadratic':
        error=1/training_examples*np.sum(0.5*(a-y_true)**2)
    elif cost_function=='CrossEntropy':
        error=1/training_examples*np.nan_to_num(np.sum(-y_true*np.log(a)-(1-y_true)*np.log(1-a)))
    else:
        raise ValueError("Invalid cost function")
    return error
def net(x, w, b): # dot product of weights and corresponding inputs
    return np.dot(w, x)+b
def sigmoid(x): # logistic activation function
    return 1.0/(1.0+np.exp(-x))
def softmax(x):
    return np.exp(x)/np.sum(np.exp(x), axis=0, keepdims=True)
def sigmoid_der(x): # Derivative of logistic activation function
    return sigmoid(x)*(1.0-sigmoid(x))
# Forward propgating function in Neural network
def forward_propagation(x,w,b, n_hidden_layers):
    nets=[None]*(number_hidden_layers+1)
    activations=[None]*(number_hidden_layers+1)
    activations_der=[None]*(number_hidden_layers+1)
    nets[0]=net(x, w[0], b[0])
    activations[0]=sigmoid(nets[0])
    activations_der[0]=sigmoid_der(nets[0])
    for k in range(n_hidden_layers-1):
        net_temp=net(activations[k], weights[k+1], bias[k+1])
        nets[k+1]=net_temp
        activations[k+1]=sigmoid(net_temp)
        activations_der[k+1]=sigmoid_der(net_temp)
    nets[-1]=net(activations[-2], weights[-1], bias[-1])
    if activation_last_layer=='Sigmoid':
        activations[-1]=sigmoid(nets[-1])
        activations_der[-1]=sigmoid_der(nets[-1])
    elif activation_last_layer=='Softmax':
        activations[-1]=softmax(nets[-1])
        activations_der[-1]=[]
    else:
        raise ValueError("Invalid activation for the last layer")
    return activations, activations_der # return activations for all layers and its derivative
# Backword propagation function in Neral network
def backpropagation(x, y, a, a_der, n_hidden_layers):
    dError_dWeights=[None]*(number_hidden_layers+1)
    dError_dWeights_bias=[None]*(number_hidden_layers+1)
    error_output=a[-1]-y
    
    if cost_function=='Quadratic':
        delta=error_output*a_der[-1] # the case for mean square error cost function
    elif cost_function=='CrossEntropy' or activation_last_layer=='Softmax':
        delta=error_output # the case for cross entropy cost function
    else:
        raise ValueError("Invalid cost function")
    dError_dWeights[-1]=np.dot(delta, a[-2].T)
    dError_dWeights_bias[-1]=np.sum(delta, axis=1, keepdims=True)
    for k in range(n_hidden_layers):
        delta=np.dot(weights[-(k+1)].T, delta)*a_der[-(k+2)]
        dError_dWeights_bias[-(k+2)]=np.sum(delta, axis=1, keepdims=True)
        if k==n_hidden_layers-1:
            dError_dWeights[-(k+2)]=np.dot(delta, x.T)
        else:
            dError_dWeights[-(k+2)]=np.dot(delta, a[-(k+3)].T)
    return dError_dWeights, dError_dWeights_bias
 # return partial derivatives of cost function with repsect to all weights and biases for all layers

In [8]:
X_test=inputs_test
labels_test=data_labels_test
labels=data_labels

## Splitting to training and validation sets (validation set is useful for hyperparameters optimisation)

In [9]:
labels=labels.reshape(-1,1)
inputs, labels=shuffle_data(inputs, labels.T)
labels=np.squeeze(labels)
X_train=inputs[:,:50000]
labels_train=labels[:50000]
X_val=inputs[:,50000:]
labels_val=labels[50000:]

## Converting labels to column vectors of size 10 (one hot encoding)

In [10]:
y_train=np.zeros((10, labels_train.shape[0]))
y_val=np.zeros((10, labels_val.shape[0]))
y_test=np.zeros((10, labels_test.shape[0]))
for i in range(labels_train.shape[0]):
    y_train[labels_train[i], i]=1
for i in range(labels_val.shape[0]):
    y_val[labels_val[i], i]=1
for i in range(labels_test.shape[0]):
    y_test[labels_test[i], i]=1
print("Original labels: {0}".format(labels_train[:10]))
print("One hot encoded labels:\n {0}".format(y_train[:, :10]))

Original labels: [4 1 9 5 0 4 6 6 1 0]
One hot encoded labels:
 [[0. 0. 0. 0. 1. 0. 0. 0. 0. 1.]
 [0. 1. 0. 0. 0. 0. 0. 0. 1. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 1. 0. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 1. 1. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0. 0. 0. 0. 0.]]


## Scaling all the pixels to unifrom range

In [11]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train.T).T
X_val = sc.transform(X_val.T).T
X_test = sc.transform(X_test.T).T

In [12]:
number_training_examples=X_train.shape[1]
number_inputs=network_architecture[0]
number_output=network_architecture[-1]
number_hidden_layers=len(network_architecture)-2
number_mini_batches=number_training_examples/mini_batch_size

## Initialize each weight using a Gaussian distribution with mean 0 and standard deviation 1 over the square root of the number of weights connecting to the same neuron.  Initialize the biases using a Gaussian distribution with mean 0 and standard deviation 1

In [13]:
weights=[]
bias=[]
for i in range(number_hidden_layers+1):
    weights.append(np.random.rand(network_architecture[i+1],network_architecture[i])/np.sqrt(network_architecture[i]))
    bias.append(np.random.rand(network_architecture[i+1], 1))

## Main block. Here we go through epochs and within each epoch we run through all mini-batches and update weights and biases for each mini-batch. Within each epoch cost and accuracy are calculated for training, validation and test sets. Accuracies and costs are plotted. Accuracy of test set is additionally printed

In [14]:
out_test=[None]*(number_hidden_layers+1)
cost_train=np.array([])
cost_val=np.array([])
cost_test=np.array([])
accuracy_train=np.array([])
accuracy_val=np.array([])
accuracy_test=np.array([])
iteration=np.array([])

# Initializing graph to plot cost function and accuracy
%matplotlib tk
plt.ion()
fig1, ax1=plt.subplots()
fig2, ax2=plt.subplots()

ax1.set_xlabel("Epochs")
ax1.set_ylabel("Cost")

ax2.set_xlabel("Epochs")
ax2.set_ylabel("Accuracy")
label_added=False
# Main loop
for epoch in range(epochs):
    # Splitting training set into minibatches
    X_train_shuffled, y_train_shuffled=shuffle_data(X_train, y_train)
    X_train_minibatches=[X_train_shuffled[:, i:i+mini_batch_size] for i in range(0, number_training_examples, mini_batch_size)]
    y_train_minibatches=[y_train_shuffled[:, i:i+mini_batch_size] for i in range(0, number_training_examples, mini_batch_size)]
    
    # Updaiting weights and biases for each mini-batch
    for i in range(len(X_train_minibatches)):            
        # Feedforward step
        activations, activations_der=forward_propagation(X_train_minibatches[i],weights,bias, number_hidden_layers)
        ############################################
        # Backpropagation step
        dError_dWeights, dError_dWeights_bias=backpropagation(X_train_minibatches[i], y_train_minibatches[i], activations, activations_der, number_hidden_layers)
        
        # Updating weights using also regularization parameters
        for k in range(number_hidden_layers+1):
            weights[k]=(1-learning_rate*regularization/number_training_examples)*weights[k]-1/mini_batch_size*learning_rate*dError_dWeights[k]
            #weights[k]=weights[k]-1/mini_batch_size*learning_rate*dError_dWeights[k]
            bias[k] =bias[k]-1/mini_batch_size*learning_rate*dError_dWeights_bias[k]
    # Predicted labels for train, validation and test sets
    y_pred_train, temp=forward_propagation(X_train, weights, bias, number_hidden_layers)
    y_pred_val, temp=forward_propagation(X_val, weights, bias, number_hidden_layers)
    y_pred_test, temp=forward_propagation(X_test, weights, bias, number_hidden_layers)
    labels_predicted_train=np.argmax(y_pred_train[-1], axis=0) # predicted digits for train set
    labels_predicted_val=np.argmax(y_pred_val[-1], axis=0) # predicted digits for validation set
    labels_predicted_test=np.argmax(y_pred_test[-1], axis=0) # predicted digits for test set
    
    # Calculating amount of correctly predicted digits
    correctly_predicted_train=np.sum(labels_predicted_train==labels_train) 
    correctly_predicted_val=np.sum(labels_predicted_val==labels_val)
    correctly_predicted_test=np.sum(labels_predicted_test==labels_test)
    
    # Plotting
    cost_train=np.append(cost_train, cost_fn(y_train, y_pred_train[-1], len(labels_train)))
    cost_val=np.append(cost_val, cost_fn(y_val, y_pred_val[-1], len(labels_val)))
    cost_test=np.append(cost_test, cost_fn(y_test, y_pred_test[-1], len(labels_test)))
    accuracy_train=np.append(accuracy_train, correctly_predicted_train/len(labels_train))
    accuracy_val=np.append(accuracy_val, correctly_predicted_val/len(labels_val))
    accuracy_test=np.append(accuracy_test, correctly_predicted_test/len(labels_test))
    iteration=np.append(iteration, epoch) 
    if not label_added:
        ax1.plot(iteration, cost_train, label="Train", color='b')
        ax1.plot(iteration, cost_val, label="Validation", color='m')
        ax1.plot(iteration, cost_test, label="Test", color='g')
        ax2.plot(iteration, accuracy_train, label="Train", color='r')
        ax2.plot(iteration, accuracy_val, label="Validation", color='k')
        ax2.plot(iteration, accuracy_test, label="Test", color='c')
        label_added=True
    else:
        ax1.plot(iteration, cost_train, color='b')
        ax1.plot(iteration, cost_val, color='m')
        ax1.plot(iteration, cost_test, color='g')
        ax2.plot(iteration, accuracy_train, color='r')
        ax2.plot(iteration, accuracy_val, color='k')
        ax2.plot(iteration, accuracy_test, color='c')
    fig2.canvas.draw()
    fig2.canvas.flush_events()
    fig1.canvas.draw()
    fig1.canvas.flush_events()

    ax1.legend()
    ax2.legend()
    print("Accuracy for test set. Epoch {0}: {1} / {2}".format(epoch, correctly_predicted_test, len(labels_test)))


Accuracy for test set. Epoch 0: 8970 / 10000
Accuracy for test set. Epoch 1: 9390 / 10000
Accuracy for test set. Epoch 2: 9528 / 10000
Accuracy for test set. Epoch 3: 9595 / 10000
Accuracy for test set. Epoch 4: 9576 / 10000
Accuracy for test set. Epoch 5: 9607 / 10000
Accuracy for test set. Epoch 6: 9655 / 10000
Accuracy for test set. Epoch 7: 9649 / 10000
Accuracy for test set. Epoch 8: 9674 / 10000
Accuracy for test set. Epoch 9: 9675 / 10000
Accuracy for test set. Epoch 10: 9700 / 10000
Accuracy for test set. Epoch 11: 9701 / 10000
Accuracy for test set. Epoch 12: 9700 / 10000
Accuracy for test set. Epoch 13: 9692 / 10000
Accuracy for test set. Epoch 14: 9702 / 10000
Accuracy for test set. Epoch 15: 9707 / 10000
Accuracy for test set. Epoch 16: 9717 / 10000
Accuracy for test set. Epoch 17: 9712 / 10000
Accuracy for test set. Epoch 18: 9709 / 10000
Accuracy for test set. Epoch 19: 9724 / 10000
Accuracy for test set. Epoch 20: 9714 / 10000
Accuracy for test set. Epoch 21: 9725 / 1000